<a href="https://colab.research.google.com/github/YoTaSo/datascience/blob/main/deeplearning/pytorch_autograd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Graphe de calcul, autograd et modules
import numpy as np
import pandas as pd
import torch
from torch.autograd import Function
from torch.autograd import gradcheck
import torch.nn.functional as F
from torch import nn
from torch import optim
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:

#I use the boston housing data to build the model on
url='https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data'
df1 = pd.read_csv(url,delim_whitespace=True,header=None)
xMat=df1.iloc[:,0:13]
yRes=df1.iloc[:,13]
augX=[]
for i in range(xMat.shape[0]):
  b=np.concatenate([[1],xMat.values[i]])
  augX.append(b)
augX = np.asarray(augX)
x_train_tensor = torch.from_numpy(augX).float().to(device)
y_train_tensor = torch.from_numpy(yRes.values).float().to(device)
y_train_tensor = y_train_tensor.view(-1,1)

In [ ]:

class Context:
    """Very simplified context object"""
    def __init__(self):
        self._saved_tensors = ()
    def save_for_backward(self, *args):
        self._saved_tensors = args
    @property
    def saved_tensors(self):
        return self._saved_tensors
        from torch.autograd import Function

class Linear(Function):
  @staticmethod
  def forward(ctx, x,w):
    ctx.save_for_backward(x,w)
    return x.mm(w.t())
    
  @staticmethod
  def backward(ctx, grad_output):
    x,w = ctx.saved_tensors
    grad_x = grad_w = None
    grad_x = grad_output.mm(w)
    grad_w = grad_output.t().mm(x)
    #grad_b = grad_output.sum(0).squeeze(0)

    return grad_x, grad_w

In [ ]:
linear = Linear()
ctx = Context()

In [ ]:

linear_check = Linear.apply
x = torch.randn(506,14,requires_grad=True,dtype=torch.float64,device=device)
w = torch.randn(1,14,requires_grad=True,dtype=torch.float64,device=device)
torch.autograd.gradcheck(linear_check,(x,w))

True

In [ ]:
#regression with gradient descent
torch.random.manual_seed(3360)
w = torch.randn(1,14,requires_grad=True,dtype=torch.float,device=device)
learning_rate = 1e-9
for t in range(10000):
  y_pred = linear.forward(ctx,x_train_tensor,w)
  loss = (y_pred - y_train_tensor).pow(2).mean()
  if t%50==0:
    print(t, loss.item())
  loss.backward()
  with torch.no_grad():
    w -= learning_rate * w.grad
  w.grad.zero_()

In [ ]:
############################second part of the TP

class MyNeuralNet(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.hidden = nn.Linear(14, 10)
    self.output = nn.Linear(10, 1)
  
  def forward(self, x):
    x = self.hidden(x)
    x = torch.tanh(x)
    x = self.output(x)
    return x


In [ ]:

torch.manual_seed(42)

# Now we can create a model and send it at once to the device
model = MyNeuralNet().to(device)
# We can also inspect its parameters using its state_dict
#print(model.state_dict())

lr = 1e-5
n_epochs = 5000

loss_fn = torch.nn.MSELoss(reduction='mean')
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

for t in range(n_epochs):
    # to initilize model
    model.train()

    # to get prediction by model
    yhat = model(x_train_tensor)
    
    loss = loss_fn(y_train_tensor, yhat)
    if t%100==0:
      print(t, loss.item())
    loss.backward()

    optimizer.step()
    optimizer.zero_grad()
    

In [ ]:
#########################################
#this block didn't work
#mynet = nn.Sequential(
#    nn.Linear(14, 10),
#    nn.Tanh(),
#    nn.Linear(10, 1))
#
#mynet.to(device)
#
#lr = 1e-4
#n_epochs = 5000
#
#loss_fn = torch.nn.MSELoss(reduction='mean')
#optimizer = torch.optim.SGD(model.parameters(), lr=lr)
#
#for t in range(n_epochs):
#    # to initilize model
#    #mynet.train()
#
#   # to get prediction by model
#    yhat = mynet(x_train_tensor)
#    
#    loss = loss_fn(yhat,y_train_tensor)
#    if t%100==0:
#      print(t, loss.item())
#    mynet.zero_grad()
#    loss.backward()
#    optimizer.step()
#    optimizer.zero_grad()
####################################    

In [ ]:
#this one worked
#with sequential
mynet = nn.Sequential(
    nn.Linear(14, 10),
    nn.Tanh(),
    nn.Linear(10, 1))

mynet.to(device)

loss_fn = torch.nn.MSELoss(reduction='mean')
learning_rate = 1e-4
for t in range(5000):
    y_pred = mynet(x_train_tensor)
    loss = loss_fn(y_pred, y_train_tensor)
    if t % 100 == 99:
        print(t, loss.item())
    mynet.zero_grad()
    loss.backward()
    #I don't know why optimizer.step() didn't work
    with torch.no_grad():
        for param in mynet.parameters():
            param -= learning_rate * param.grad